# Mga Generative na Network

Ang Recurrent Neural Networks (RNNs) at ang mga gated cell variant nito tulad ng Long Short Term Memory Cells (LSTMs) at Gated Recurrent Units (GRUs) ay nagbibigay ng mekanismo para sa pagmomodelo ng wika, ibig sabihin, kaya nilang matutunan ang pagkakasunod-sunod ng mga salita at magbigay ng prediksyon para sa susunod na salita sa isang sequence. Dahil dito, magagamit natin ang RNNs para sa **mga generative na gawain**, tulad ng ordinaryong pagbuo ng teksto, pagsasalin ng makina, at maging sa paglalagay ng caption sa mga imahe.

Sa arkitektura ng RNN na tinalakay natin sa nakaraang unit, ang bawat RNN unit ay gumagawa ng susunod na hidden state bilang output. Gayunpaman, maaari rin tayong magdagdag ng isa pang output sa bawat recurrent unit, na magpapahintulot sa atin na mag-output ng isang **sequence** (na may parehong haba sa orihinal na sequence). Bukod pa rito, maaari tayong gumamit ng mga RNN unit na hindi tumatanggap ng input sa bawat hakbang, at sa halip ay gumagamit lamang ng isang initial state vector, at pagkatapos ay gumagawa ng isang sequence ng mga output.

Sa notebook na ito, magpo-focus tayo sa mga simpleng generative na modelo na tumutulong sa atin na bumuo ng teksto. Para sa pagiging simple, magtatayo tayo ng **character-level network**, na bumubuo ng teksto letra kada letra. Sa panahon ng training, kailangan nating kumuha ng isang text corpus, at hatiin ito sa mga sequence ng letra.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Pagbuo ng bokabularyo ng karakter

Para makabuo ng generative network sa antas ng karakter, kailangan nating hatiin ang teksto sa mga indibidwal na karakter sa halip na mga salita. Magagawa ito sa pamamagitan ng pagde-define ng ibang tokenizer:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Tingnan natin ang halimbawa kung paano natin mai-encode ang teksto mula sa ating dataset:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Pagsasanay ng isang generative RNN

Ang paraan ng pagsasanay natin sa RNN upang makabuo ng teksto ay ang sumusunod. Sa bawat hakbang, kukuha tayo ng isang sequence ng mga karakter na may haba na `nchars`, at hihilingin sa network na bumuo ng susunod na output na karakter para sa bawat input na karakter:

![Larawan na nagpapakita ng halimbawa ng RNN na bumubuo ng salitang 'HELLO'.](../../../../../translated_images/tl/rnn-generate.56c54afb52f9781d.webp)

Depende sa aktwal na sitwasyon, maaaring gusto rin nating isama ang ilang espesyal na karakter, tulad ng *end-of-sequence* `<eos>`. Sa ating kaso, nais lamang nating sanayin ang network para sa walang katapusang pagbuo ng teksto, kaya't itatakda natin ang laki ng bawat sequence na maging katumbas ng `nchars` na mga token. Dahil dito, ang bawat halimbawa ng pagsasanay ay binubuo ng `nchars` na mga input at `nchars` na mga output (na siyang input sequence na inilipat ng isang simbolo sa kaliwa). Ang minibatch ay binubuo ng ilang ganitong mga sequence.

Ang paraan ng pagbuo natin ng mga minibatch ay ang pagkuha ng bawat teksto ng balita na may haba na `l`, at pagbuo ng lahat ng posibleng input-output na kombinasyon mula rito (magkakaroon ng `l-nchars` na ganitong mga kombinasyon). Ang mga ito ay bubuo ng isang minibatch, at ang laki ng mga minibatch ay mag-iiba sa bawat hakbang ng pagsasanay.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Ngayon, magde-define tayo ng generator network. Maaari itong ibase sa anumang recurrent cell na tinalakay natin sa nakaraang unit (simple, LSTM, o GRU). Sa ating halimbawa, gagamit tayo ng LSTM.

Dahil ang network ay tumatanggap ng mga karakter bilang input, at maliit lamang ang laki ng bokabularyo, hindi na natin kailangan ng embedding layer; ang one-hot-encoded na input ay maaaring direktang ipadala sa LSTM cell. Gayunpaman, dahil mga numero ng karakter ang ipinapasa bilang input, kailangan nating i-one-hot-encode ang mga ito bago ipadala sa LSTM. Ginagawa ito sa pamamagitan ng pagtawag sa `one_hot` function sa panahon ng `forward` pass. Ang output encoder ay magiging isang linear layer na magko-convert ng hidden state sa one-hot-encoded na output.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Sa panahon ng pagsasanay, nais nating makapag-sample ng mga nilikhang teksto. Upang magawa ito, magde-define tayo ng `generate` na function na maglalabas ng output na string na may habang `size`, simula sa paunang string na `start`.

Ganito ang paraan ng paggana nito. Una, ipapasa natin ang buong start string sa network, at kukunin ang output state na `s` at ang susunod na hinulaang karakter na `out`. Dahil ang `out` ay naka-one-hot encode, kukunin natin ang `argmax` upang makuha ang index ng karakter na `nc` sa bokabularyo, at gagamitin ang `itos` upang malaman ang aktwal na karakter at idagdag ito sa listahan ng mga karakter na `chars`. Ang prosesong ito ng pagbuo ng isang karakter ay inuulit nang `size` na beses upang makabuo ng kinakailangang bilang ng mga karakter.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Ngayon, simulan na natin ang pagsasanay! Ang training loop ay halos pareho sa lahat ng ating mga naunang halimbawa, ngunit sa halip na accuracy, magpi-print tayo ng mga halimbawa ng generated text tuwing 1000 epochs.

Kailangang bigyan ng espesyal na pansin ang paraan ng pagkalkula ng loss. Kailangan nating kalkulahin ang loss gamit ang one-hot-encoded na output na `out`, at ang inaasahang text na `text_out`, na siyang listahan ng mga character indices. Sa kabutihang-palad, ang function na `cross_entropy` ay inaasahan ang unnormalized network output bilang unang argumento, at ang class number bilang pangalawa, na eksaktong tumutugma sa kung ano ang mayroon tayo. Gumagawa rin ito ng awtomatikong pag-average batay sa minibatch size.

Nililimitahan din natin ang pagsasanay sa pamamagitan ng `samples_to_train` na mga sample, upang hindi masyadong magtagal ang proseso. Hinihikayat namin kayong mag-eksperimento at subukang magpatakbo ng mas mahabang pagsasanay, posibleng para sa ilang epochs (kung saan kakailanganin ninyong gumawa ng isa pang loop sa paligid ng code na ito).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Ang halimbawang ito ay nakabubuo na ng medyo maayos na teksto, ngunit maaari pa itong mapahusay sa iba't ibang paraan:

* **Mas maayos na minibatch generation**. Ang paraan ng paghahanda natin ng data para sa training ay sa pamamagitan ng pagbuo ng isang minibatch mula sa isang sample. Hindi ito ideal, dahil ang mga minibatch ay magkakaiba ang laki, at ang ilan sa mga ito ay hindi pa nga maaaring mabuo, dahil mas maliit ang teksto kaysa sa `nchars`. Bukod dito, ang maliliit na minibatch ay hindi sapat na nagagamit ang GPU. Mas mainam na kumuha ng isang malaking bahagi ng teksto mula sa lahat ng mga sample, pagkatapos ay bumuo ng lahat ng input-output na pares, i-shuffle ang mga ito, at bumuo ng mga minibatch na magkakapareho ang laki.

* **Multilayer LSTM**. Makatuwiran na subukan ang 2 o 3 layer ng LSTM cells. Tulad ng nabanggit natin sa nakaraang unit, bawat layer ng LSTM ay kumukuha ng partikular na mga pattern mula sa teksto, at sa kaso ng character-level generator, maaari nating asahan na ang mas mababang LSTM level ay responsable sa pagkuha ng mga pantig, at ang mas mataas na mga level - para sa mga salita at kombinasyon ng mga salita. Madali itong maipatupad sa pamamagitan ng pagpapasa ng number-of-layers parameter sa LSTM constructor.

* Maaari mo ring subukang mag-eksperimento gamit ang **GRU units** at tingnan kung alin ang mas mahusay ang performance, pati na rin ang **iba't ibang laki ng hidden layer**. Ang sobrang laki ng hidden layer ay maaaring magresulta sa overfitting (halimbawa, matututunan ng network ang eksaktong teksto), at ang mas maliit na laki ay maaaring hindi makabuo ng magandang resulta.


## Malambot na pagbuo ng teksto at temperatura

Sa nakaraang depinisyon ng `generate`, palagi nating pinipili ang karakter na may pinakamataas na posibilidad bilang susunod na karakter sa nabubuong teksto. Dahil dito, madalas na "umuulit" ang teksto sa parehong mga pagkakasunod-sunod ng karakter, tulad ng sa halimbawang ito:
```
today of the second the company and a second the company ...
```

Gayunpaman, kung titingnan natin ang distribusyon ng posibilidad para sa susunod na karakter, maaaring ang pagkakaiba sa pagitan ng ilang pinakamataas na posibilidad ay hindi ganoon kalaki, halimbawa, ang isang karakter ay maaaring may posibilidad na 0.2, habang ang isa pa ay 0.19, at iba pa. Halimbawa, kapag naghahanap ng susunod na karakter sa pagkakasunod-sunod na '*play*', ang susunod na karakter ay maaaring pantay na maging espasyo o **e** (tulad ng sa salitang *player*).

Ito ay nagdadala sa atin sa konklusyon na hindi palaging "makatarungan" na piliin ang karakter na may mas mataas na posibilidad, dahil ang pagpili sa pangalawang pinakamataas ay maaari pa ring magresulta sa makabuluhang teksto. Mas matalino ang **pag-sample** ng mga karakter mula sa distribusyon ng posibilidad na ibinigay ng output ng network.

Ang pag-sample na ito ay maaaring gawin gamit ang `multinomial` na function na nagpapatupad ng tinatawag na **multinomial distribution**. Ang isang function na nagpapatupad ng ganitong **malambot** na pagbuo ng teksto ay nakasaad sa ibaba:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Inilunsad namin ang isa pang parameter na tinatawag na **temperature**, na ginagamit upang ipakita kung gaano kahigpit dapat tayong sumunod sa pinakamataas na posibilidad. Kung ang temperature ay 1.0, gumagawa tayo ng patas na multinomial sampling, at kapag ang temperature ay umabot sa infinity - nagiging pantay-pantay ang lahat ng posibilidad, at random tayong pumipili ng susunod na karakter. Sa halimbawa sa ibaba, mapapansin natin na nagiging walang kahulugan ang teksto kapag masyadong tinaasan ang temperature, at nagmumukhang "cycled" na mahigpit na binuong teksto kapag mas malapit ito sa 0.



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na dulot ng paggamit ng pagsasaling ito.
